In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import hvplot.pandas

In [ ]:
fn = "../subworkflows/pypsa-earth-sec/data/industrial_database.csv"

In [ ]:
idb = pd.read_csv(fn, index_col=0)

In [ ]:
# Filter for country MA
idb = idb[idb.index == "MA"]

In [ ]:
idb #.head()

In [ ]:
px.scatter_mapbox(
    idb,
    lat="y",
    lon="x",
    mapbox_style="carto-positron",
    color="technology",
    #size="capacity",
    # size
    #hover_name="name",
    zoom=4,
    height=600,
)

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import cartopy.crs as ccrs
import cartopy

HV plot

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
colors = {
    "Cement": "#80c944",
    "Electric arc": "red",
    "HVC": "purple",
}

In [ ]:
df = gpd.GeoDataFrame(idb, geometry=gpd.points_from_xy(idb["x"], idb["y"]), crs="EPSG:4326")
crs = ccrs.AlbersEqualArea()

#df = df.cx[-12:40, 35:72]
df = df.to_crs(crs.proj4_init)

fig, ax = plt.subplots(figsize=(9.5, 9.5), subplot_kw={"projection": crs})

ax.add_feature(cartopy.feature.COASTLINE.with_scale("50m"), linewidth=0.2, zorder=2)
ax.add_feature(cartopy.feature.BORDERS.with_scale("50m"), linewidth=0.2, zorder=2)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.LAND, edgecolor="black")

df.plot(
    ax=ax,
    column="technology",
    markersize=df["capacity"] / 35,
    alpha=0.75,
    legend=True,
    cmap=mcolors.ListedColormap(
        pd.Series(df.technology.unique()).sort_values().map(colors).values
    ),
    legend_kwds=dict(title="Technology (size ~ capacity)", frameon=True),
)
#cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, zoom=10)

ax.set_frame_on(False)
ax.set_facecolor("white")
#plt.savefig(OUTPUT + "/powerplants.pdf", bbox_inches="tight")